In [1]:
import os 
import pickle as pkl
import numpy as np
import math
from matplotlib import pyplot as plt 

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [2]:
import sys
sys.path.append('/nas/home/drizzle0171/motion-generation/MotionGPDiffusion')

from LPM.model_final import (
    DiffusionUNetLegacy,
)

import torch
import torch.nn as nn
model = DiffusionUNetLegacy(
            name                 = 'unet',
            length               = 60, 
            dims                 = 1,
            n_in_channels        = 1,
            n_base_channels      = 128,
            n_emb_dim            = 128,
            n_cond_dim           = 0,
            n_time_dim           = 0,
            n_enc_blocks         = 7, # number of encoder blocks
            n_groups             = 16, # group norm paramter
            n_heads              = 4, # number of heads in QKV attention
            actv                 = nn.SiLU(),
            kernel_size          = 3, # kernel size (3)
            padding              = 1, # padding size (1)
            use_attention        = False,
            skip_connection      = True, # additional skip connection
            chnnel_multiples     = [1,2,2,2,4,4,8],
            updown_rates         = [1,1,2,1,2,1,2],
            use_scale_shift_norm = True,
            device               = 'cuda',
        ) # input:[B x C x L] => output:[B x C x L]

model.load_state_dict(torch.load('/nas/home/drizzle0171/motion-generation/MotionGPDiffusion/LPM/result/original-b7-lr1e-4-schdCS-wd0.1-dropout-norm-length60/ckpt_226000_0.3406183421611786.pt')['model_state_dict'])

cls_value = torch.Tensor([0.03, 0.12,
                        0.21, 0.3,
                        0.39, 0.48,
                        0.57, 0.66,
                        0.8, 1.0]).cuda()

model.eval()

train_data = np.load('/nas/home/drizzle0171/motion-generation/MotionGPDiffusion/LPM/train_big_norm_60.npy', allow_pickle=True)
train_data = train_data.item()
train_x_0 = train_data['x_0']

/home/drizzle0171/miniconda3/envs/gluegen/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
num_sample = 100

with open("/nas/home/drizzle0171/motion-generation/MotionGPDiffusion/dataset/LAFAN/lafan_60_train_data.pkl", 'rb') as f:
    data = pkl.load(f)
    
times = np.arange(60)
new_data = (data['rot_6d']- np.mean(data['rot_6d'], axis=(0,1)))/np.std(data['rot_6d'], axis=(0,1))
data_sample = torch.Tensor(new_data[20000:20000+num_sample,:,:,:]).reshape(num_sample, 60, 132).cuda()

In [10]:
val_data = np.load('/nas/home/drizzle0171/motion-generation/MotionGPDiffusion/LPM/val_norm_60.npy', allow_pickle=True)
val_data = val_data.item()
val_x_0 = val_data['x_0']
val_label = val_data['real_param']
val_x_0 = (val_x_0 - train_x_0.mean()) / math.sqrt(train_x_0.var())

In [13]:
data_sample = data_sample.permute(0, 2, 1).reshape(num_sample*132, 60)[:, None]

with torch.no_grad():
    output = model(data_sample)
    
_, pred_idx = torch.max(output.data, 1)
output = pred_idx
pred = cls_value[pred_idx]

# 각 length parameter마다 예측한 갯수
for i in [0.03, 0.12, 0.21, 0.3, 0.39, 0.48, 0.57, 0.66, 0.8, 1.0]:
    print(i, ' -> ',sum(pred == i).detach().cpu().numpy())

torch.Size([13200, 1, 60])
tensor([0.1200, 0.1200, 0.1200,  ..., 0.1200, 0.1200, 0.1200], device='cuda:0')


In [ ]:
params = [0.03, 0.12, 0.21, 0.3, 0.39, 0.48, 0.57, 0.66, 0.8, 1.0]

for i in range(10):
    fig, axes= plt.subplots(1,2, figsize=(40, 15))
    fig.tight_layout()
    param = pred == params[i]
    cnt = 0
    
    for j in range(len(param)):
        if param[j]:
            axes[0].plot(times, data_sample[j].detach().cpu().numpy().squeeze(), color='royalblue', alpha=0.6, linewidth=3)
            axes[0].set_title(f'Predicted {params[i]}', fontsize=50, pad=25)
            axes[0].set_ylim(-3, 3)
            axes[0].tick_params(axis="x", labelsize=25)
            axes[0].tick_params(axis="y", labelsize=25)
            cnt += 1
        if cnt > 20:
            break
            cnt = 0
            
    for k in range(5, 20):
        m = i * 100
        axes[1].plot(times, val_x_0[m+k, :, :].detach().cpu().numpy().squeeze(), color='salmon', linewidth=3)
        axes[1].set_title(f'Trajectory when length parameter {params[i]}', fontsize=50, pad=25)
        axes[1].set_ylim(-3, 3)
        axes[1].tick_params(axis="x", labelsize=25)
        axes[1].tick_params(axis="y", labelsize=25)
        
    plt.savefig(f'/nas/home/drizzle0171/motion-generation/MotionGPDiffusion/LPM/result/f60_len_{str(params[i])}.png')
    # plt.clf()
            